In [ ]:
import json
import os

In [ ]:
path = "/content/drive/MyDrive/WebAI/Notebooks/data/clean_data"
dir_list = os.listdir(path)

In [ ]:
# Recursion to check properties of a child element
# - focus
def check_children(index, children, action, html):
  #print(f'{index} - {children}')
  action_performed = None

  tag, element = recreate_html(children)
  # pretty:
  #html += "    "*index + element + "\n"
  html += element
  if 'children' in children:
    #print('going deeper')
    # Branch splits happens here. Todo: potentially split snippets at this level
    for child in children['children']:
      html, action_received = check_children(index+1, child, action, html)
      if action_received is not None:
        action_performed = action_received

  if 'focused' in children:
    ref = children['ref']
    if action is None:
      action_type = "none"
    else:
      action_type = action['type']

    #print(f'yes focused at index:{index} for ref:{ref} -> {children}')
    action_performed = {
        'action': action_type,
        'ref': ref
    }
    #print(f'action_performed: {action_performed}')

  # closing tag
  if tag == "INPUT_checkbox":
    html += "</input>"
  else:
    html += "</" + tag + ">"

  # closing tag pretty. todo: add matching correct tag like input type=checkbox
  #html += "    "*index + "</" + tag + ">\n"
  return html, action_performed


# fields we can drop when re-creating the HTML element
drop_fields = ['children', 'bgColor', 'width', 'height', 'focused', 'fgColor', 'right', 'top', 'left', 'bottom']

# Recreate HTML elements
def recreate_html(child):
  fields = child.copy()
  for f in drop_fields:
    if f in fields:
      del fields[f]

  tag = fields.pop('tag')
  # parse input type checkbox
  if tag == 'INPUT_checkbox':
    element = '<input type="checkbox" '
  else:
    element = '<' + tag + " "
  for k, value in fields.items():
    #print(k)
    if value != '':
      element += k + '="' + str(value) + '" '
  element = element[:-1] + ">"

  #print(f'element: {element}')
  return tag, element

In [ ]:

def process_data_entry(data):

    utterance = data['utterance']
    #print(f'utterance: {utterance}')
    state = data['states']
    #print(state)
    #print(len(state))

    actions = [] # list of the actions done at each state
    htmls = [] # list of different html so we can monitor changes in the episode. They should be the same but we never know

    for s in state:

      action = s['action']
      dom = s['dom']
      #time = s['time']


      # loop to get over all children elements
      html, action_performed = check_children(0, dom, action, "")
      htmls.append(html)
      actions.append(action_performed)

      #print(f'final html: {html}')
      #print(f'action performed: {action_performed}')
      #print('------------------------------')

    return utterance, actions, htmls


  #break


In [ ]:

# create the input data by aggregating the parsed data
# Each html content is map to the action to be performed in order to arrive to the next state
def generate_training_data_state(utterance, htmls, actions):

    if len(htmls) != len(actions):
        print(f'html {len(htmls)} does not have the same size as actions {len(actions)}')

    if len(htmls) == 0:
        print("html has len 0")
        return None, None
    if len(actions) == 0:
        print("actions has len 0")
        return None, None


    xs = []
    ys = []

    try:

        # initial step:
        xs.append(utterance + ' ' + htmls[0])
        ys.append([actions[1]['action'], actions[1]['ref']])

        i = 1;
        # last element is a terminal state, don't include it.
        while i < len(htmls)-1:
            s_actions = actions[1:i+1]
            s_actions.reverse()
            s_action = ''.join(str(x) for x in s_actions).replace("'", "")

            x = s_action + ' ' + utterance + ' ' + htmls[i]
            xs.append(x)

            action = actions[i+1]['action']
            ref = actions[i+1]['ref']
            ys.append([action, ref])

            i += 1

        return xs, ys

    except:
        return None, None


In [ ]:
xs, ys = [], []


In [ ]:


# Read and process the entire dataset
dir_list = os.listdir(path)
dir_list.remove(".DS_Store")
print(dir_list)

for dir in dir_list:
    subdir_list = os.listdir(path + "/" + dir)
    if ".DS_Store" in subdir_list:
        subdir_list.remove(".DS_Store")
    print(subdir_list)

    for sub_entry in subdir_list:
        if "." not in subdir_list:
            subsubdir_list = os.listdir(path + "/" + dir + "/" + sub_entry)
            if ".DS_Store" in subsubdir_list:
                subsubdir_list.remove(".DS_Store")

            for subsubentry in subsubdir_list:
                print(subsubentry)
                sample_file = open(path + "/" + dir + "/" + sub_entry + "/" + subsubentry)
                data = json.load(sample_file)

                utterance, actions, htmls = process_data_entry(data)
                x, y = generate_training_data_state(utterance, htmls, actions)
                if x is None or y is None:
                    continue
                xs.append(x)
                ys.append(y)


        else:
            sample_file = open(path + "/" + dir + "/" + sub_entry)
            data = json.load(sample_file)

            utterance, actions, htmls = process_data_entry(data)
            x, y = generate_training_data_state(utterance, htmls, actions)
            if x is None or y is None:
                continue
            xs.append(x)
            ys.append(y)



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
social-media_33IZTU6J82VGVONUTLS4YWBG0YYXS8_d1.json
social-media_3KYQYYSHYW1NZQQKV4AJKKYNNRADOG_d6.json
social-media_3I2PTA7R3UOYY89A2YOAD2ODM1PKQ8_d7.json
social-media_3WQQ9FUS6BORZ15KARUHS8KN3ITB8Z_d6.json
social-media_31JLPPHS2VOPHYON2F477BCBWEF3O2_d4.json
social-media_34S9DKFK74J8FGLZQLXKIG8JF54YNN_d4.json
social-media_3YDGXNSEO0O7E2KBVCKUKUEHV9848M_d1.json
social-media_3JMSRU9HQJONHGDHSB7HLUAGJW2EV6_d2.json
social-media_3AQF3RZ559CU84YCMEJFRKFHE4WF64_d5.json
social-media_3N2BF7Y2VROGB3AA4KEWJ1OGKJAHM0_d9.json
social-media_3DZQRBDBSM94FQGG9WLGRGMCT3J3S0_d4.json
social-media_3C6FJU71TRNLA658NM96WB0DN1QUY4_d2.json
social-media_39N5ACM9HFHTH0P4YOIWNOKN6BNP9O_d6.json
social-media_39GXDJN2OU868F3QF8LHTKEU6AA8V3_d2.json
social-media_3IX2EGZR7CD3ZPISXSDYHDY51FORJA_d3.json
social-media_3NQL1CS15S2LNLU8SCJD6T0QUAVVYJ_d5.json
social-media_3SNVL38CI5MU47ZRXGNFTXZWIMLKC0_d6.json
html has len 0
social-media_3EJPLAJKEN

In [ ]:
for entry in xs[0]:
  print(entry)

for entry in ys[0]:
  print(entry)

Expand the section below and click submit. <BODY ref="1"><DIV id="wrap" ref="2"><DIV id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="3"><H3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="4"><t ref="-1" text="Section #9"></t></H3><P id="ui-id-3" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons" ref="5"><BUTTON id="subbtn" classes="secondary-action" ref="6" text="Submit"></BUTTON></P></DIV></DIV></BODY>
{action: mousedown, ref: 1} Expand the section below and click submit. <BODY ref="1"><DIV id="wrap" ref="2"><DIV id="area" classes="ui-accordion ui-widget ui-helper-reset" ref="3"><H3 id="ui-id-1" classes="ui-accordion-header ui-corner-top ui-accordion-header-collapsed ui-corner-all ui-state-default ui-accordion-icons ui-state-hover" ref="4" recordingTarget="True"><t ref="-2" text="Section #9"></t></H3><P i

In [ ]:
print(len(xs))
print(len(ys))

16794
16794


In [ ]:
import pandas as pd

In [ ]:
data = {
    "episodes": xs,
    "actions": ys
}
dataframe = pd.DataFrame(data)

In [ ]:
dataframe.head()

,episodes,actions
0,[Expand the section below and click submit. <B...,"[[mousedown, 1], [mousedown, 1], [mouseup, 4],..."
1,[Expand the section below and click submit. <B...,"[[mousedown, 1], [mousedown, 1], [mouseup, 4],..."
2,[Expand the section below and click submit. <B...,"[[mousedown, 1], [mousedown, 1], [mouseup, 4],..."
3,[Expand the section below and click submit. <B...,"[[mousedown, 8], [mousedown, 8], [mouseup, 1],..."
4,[Expand the section below and click submit. <B...,"[[mousedown, 5], [mousedown, 5], [mouseup, 1],..."


In [ ]:
# todo: Save data into sample_data and then export it to HuggingFace

# save:
dataframe.to_csv("/content/sample_data/miniwob_episodes.csv", encoding='utf-8', index=False)

In [ ]:
# upload to the hub
dataset = load_dataset("csv", data_files="/content/sample_data/miniwob_episodes.csv")

Computing checksums: 100%|##########| 1/1 [00:25<00:00, 25.42s/it]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-178c0075fd5f3d22/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset.push_to_hub("LucasThil/miniwobplusplus_episodes")

Pushing dataset shards to the dataset hub:   0%|          | 0/7 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

# Put all episodes together to have a 1D vector

In [ ]:
import pandas as pd

In [ ]:
# free ram
dataset = ""

df = pd.read_csv("/content/sample_data/miniwob_episodes.csv")

In [ ]:
df.columns

Index(['episodes', 'actions'], dtype='object')

In [ ]:
xs = []
ys = []

for index, row in df.iterrows():
  entry = row['episodes'].replace('[', '').replace(']', '').replace("', '{", "'Ø'{").split('Ø')
  actions = row['actions'].replace('[[', '[').replace(']]', ']').replace('"', '').replace('], [', ']Ø[').split('Ø')


  if len(entry) != len(actions):
    print('error')
    print(entry)
    print(actions)
    print(len(entry))
    print(len(actions))
    break

  i = 0
  while i < len(entry):
    xs.append(entry[i])
    ys.append(actions[i])
    i += 1

df = ""

In [ ]:
print(len(xs))
print(len(ys))

815482
815482


In [ ]:
data = {
    "episodes": xs,
    "actions": ys
}
dataframe = pd.DataFrame(data)

In [ ]:
len(dataframe.actions.unique())

2947

In [ ]:
dataframe.to_csv("/content/sample_data/miniwob_clean.csv", encoding='utf-8', index=False)

In [ ]:
dataset = load_dataset("csv", data_files="/content/sample_data/miniwob_clean.csv")

Computing checksums: 100%|##########| 1/1 [00:27<00:00, 27.74s/it]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-332aac3196dd0925/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset.push_to_hub("LucasThil/miniwobplusplus_ready")

Pushing dataset shards to the dataset hub:   0%|          | 0/7 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/117 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/117 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/117 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/117 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#from datasets import Dataset
#dataset_dict = {
#    "text": xs,
#    "action": ys
#}
#dataset = Dataset.from_dict({
#    "text": xs,
#    "action": ys
#})

# Transformers Part

In [ ]:
!pip install transformers
!huggingface-cli login
!pip install huggingface-hub
!pip install datasets
!pip install evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.7 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.c

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("LucasThil/miniwobplusplus")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: ignored

# Do Split between Train, Test and Eval

In [ ]:
from datasets import load_dataset_builder
ds_builder = load_dataset_builder("LucasThil/miniwobplusplus_ready")

In [ ]:
ds_builder.info.features

{'episodes': Value(dtype='string', id=None),
 'actions': Value(dtype='string', id=None)}

We'll split into multiple features: Text, Action, Ref

In [ ]:
from datasets import get_dataset_split_names
get_dataset_split_names("LucasThil/miniwobplusplus_ready")

['train']

In [ ]:
from datasets import load_dataset
dataset = load_dataset("LucasThil/miniwobplusplus_ready")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['episodes', 'actions'],
        num_rows: 815482
    })
})

In [ ]:
actions = []
refs = []

for row in dataset['train']['actions']:
  row = row.replace("'", "").replace('[', '').replace(']', '').split(', ')

  actions.append(row[0])
  refs.append(row[1])

In [ ]:
# now clean the text a bit
episodes = []

for row in dataset['train']['episodes']:
  episodes.append(row.replace("'", ""))

In [ ]:
import pandas as pd

# do splits
data = {
  "episodes": episodes,
  "actions": actions,
  "refs": refs
}

df = pd.DataFrame(data)


In [ ]:
print(len(df.actions.unique()))
print(len(df.refs.unique()))

8
893


In [ ]:
measurer = np.vectorize(len)
res1 = measurer(df['episodes'].values.astype(str)).max(axis=0)

In [ ]:
import numpy as np

In [ ]:
train, validate, test = np.split(df.sample(frac=1, random_state=42),
                       [int(.8*len(df)), int(.9*len(df))])

In [ ]:
train_csv = pd.DataFrame(train)
train_csv.to_csv("/content/sample_data/train.csv", index=False)

NameError: ignored

In [ ]:
validate_csv = pd.DataFrame(validate)
validate_csv.to_csv("/content/sample_data/validate.csv", index=False)

NameError: ignored

In [ ]:
test_csv = pd.DataFrame(test)
test_csv.to_csv("/content/sample_data/test.csv", index=False)

In [ ]:
from datasets import load_dataset
file_dict = {
  "train" : "/content/sample_data/train.csv",
  "test" : "/content/sample_data/test.csv",
  "validate": "/content/sample_data/validate.csv"
}
df_train = load_dataset("csv", data_files=file_dict)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#df_train.push_to_hub("LucasThil/miniwobplusplus_train")

Pushing dataset shards to the dataset hub:   0%|          | 0/6 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/82 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/82 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_train["train"]['episodes'].str.len().max()

AttributeError: ignored

# Dataset One Hot Actions

In [ ]:
!huggingface-cli login
!pip install huggingface-hub
!pip install datasets


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid.
Your token has been saved in your configured git crede

In [ ]:
from datasets import load_dataset
dataset = load_dataset("LucasThil/miniwobplusplus_train")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    validate: Dataset({
        features: ['episodes', 'actions', 'refs'],
        num_rows: 81548
    })
    train: Dataset({
        features: ['episodes', 'actions', 'refs'],
        num_rows: 652385
    })
    test: Dataset({
        features: ['episodes', 'actions', 'refs'],
        num_rows: 81549
    })
})

In [ ]:
import numpy as np

In [ ]:
chars_actions = sorted(list(set(dataset['train']['actions'] + dataset['test']['actions'] + dataset['validate']['actions'])))
action_size = len(chars_actions)
print(chars_actions)
print(action_size)

stoi = {ch:i for i, ch in enumerate(chars_actions)}
itos = {i:ch for i, ch in enumerate(chars_actions)}
print(stoi)
print(itos)

['click', 'dblclick', 'keydown', 'keypress', 'keyup', 'mousedown', 'mouseup', 'scroll']
8
{'click': 0, 'dblclick': 1, 'keydown': 2, 'keypress': 3, 'keyup': 4, 'mousedown': 5, 'mouseup': 6, 'scroll': 7}
{0: 'click', 1: 'dblclick', 2: 'keydown', 3: 'keypress', 4: 'keyup', 5: 'mousedown', 6: 'mouseup', 7: 'scroll'}


In [ ]:
actions = np.zeros(action_size)
actions.shape

(8,)

In [ ]:

def create_one_hot_actions(rows):
  actions = np.zeros((len(rows), len(chars_actions)))

  i = 0
  for row in rows['actions']:
    #sub_action = np.zeros(action_size)
    #sub_action[stoi[row]] = 1

    actions[i, stoi[row]] = 1

    i += 1

  return actions


In [ ]:
import pandas as pd

In [ ]:

def combine_datasets(dataset, actions, split):
  # combine dataset
  print(f'len actions {split} {len(actions)}')
  train_data_episodes = dataset[split]['episodes']
  print(f'len episodes {split} {len(train_data_episodes)}')
  train_data_refs = dataset[split]['refs']
  print(f'len refs {split} {len(train_data_refs)}')

  data = {
    "episodes": train_data_episodes,
    "refs": train_data_refs,
    "click": actions[:,0].astype(np.int64),
    "dblclick": actions[:,1].astype(np.int64),
    "keydown": actions[:,2].astype(np.int64),
    "keypress" : actions[:,3].astype(np.int64),
    "keyup" : actions[:,4].astype(np.int64),
    "mousedown": actions[:,5].astype(np.int64),
    "mouseup": actions[:,6].astype(np.int64),
    "scroll": actions[:,7].astype(np.int64),
  }

  train_csv = pd.DataFrame(data)
  train_csv.to_csv("/content/sample_data/" + split + ".csv", index=False)

  return train_csv

In [ ]:
train_csv = combine_datasets(dataset, create_one_hot_actions(dataset['train']), 'train')

len actions train 652385
len episodes train 652385
len refs train 652385


In [ ]:
train_csv.head()

,episodes,refs,click,dblclick,keydown,keypress,keyup,mousedown,mouseup,scroll
0,"{action: keydown, ref: 15}{action: keydown, re...",15,0,0,1,0,0,0,0,0
1,"{action: keypress, ref: 88}{action: keypress, ...",88,0,0,0,0,1,0,0,0
2,"{""Departure City"":""Tampa"",""Destination City"":""...",1,0,0,0,0,0,0,0,1
3,"{action: keydown, ref: 15}{action: keyup, ref:...",15,0,0,1,0,0,0,0,0
4,"{action: keydown, ref: 11}{action: keydown, re...",11,0,0,0,1,0,0,0,0


In [ ]:
test_csv = combine_datasets(dataset, create_one_hot_actions(dataset['test']), 'test')
val_csv = combine_datasets(dataset, create_one_hot_actions(dataset['validate']), 'validate')

len actions test 81549
len episodes test 81549
len refs test 81549
len actions validate 81548
len episodes validate 81548
len refs validate 81548


In [ ]:
from datasets import load_dataset
file_dict = {
  "train" : "/content/sample_data/train.csv",
  "test" : "/content/sample_data/test.csv",
  "validate": "/content/sample_data/validate.csv"
}
df_train = load_dataset("csv", data_files=file_dict)

Computing checksums:  33%|###3      | 1/3 [00:12<00:25, 12.89s/it]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validate split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-498c92b7f4a07a4d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
df_train.push_to_hub('LucasThil/miniwob_actions_onhot')

Pushing dataset shards to the dataset hub:   0%|          | 0/6 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/82 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/82 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]